In [1]:
import os
from langgraph.prebuilt import create_react_agent
from utils import query_iterator, query_to_pddl, run_ff

ff_loc = "/home/exale/FF-v2.3/ff"
model = "openai:gpt-4o"
api_key_path = r"../openai_key.txt"
with open(api_key_path, 'r') as f:
    key = f.read()
os.environ["OPENAI_API_KEY"] = key
with open("./blocksworld/blocksworld_domain_nl.txt", 'r') as f:
    domain_nl = f.read()

In [ ]:
agent_v_context = "The blocks are each labelled by a letter. I can only interact with blocks whose letters are vowels."
agent_v_prompt = domain_nl + agent_v_context
agent_v = create_react_agent(model=model,
                             tools=[], 
                             prompt=agent_v_prompt)

In [3]:
agent_c_context = "The blocks are each labelled by a letter. I can only interact with blocks whose letters are consonants."
agent_c_prompt = domain_nl + agent_c_context
agent_c = create_react_agent(model=model, 
                             tools=[], 
                             prompt=agent_c_prompt)

In [13]:
z_mode = "from_pddl"
if z_mode == "from_pddl":
    agent_z_prompt = "You will be provided two sets of .pddl domain and problem files with a common goal, " \
                     "produced by two agents working towards this goal. Your task as an orchestrator is to combine these into a " \
                     "single, consistent pair of domain and problem files describing the operation." \
                     "The two agents have potentially different capabilities, and you should distinguish between actions of one agent" \
                     "versus the other."
elif z_mode == "from_context":
    agent_z_prompt = "..." # Explanation of context, agent abilities, and that a goal will be given, for which the agent should construct pddls.
agent_z = create_react_agent(model=model, 
                             tools=[], 
                             prompt=agent_z_prompt)

In [ ]:
query_mode = "pddl"
task_query = {"pddl": "\nGenerate the contents for a domain.pddl and a problem.pddl file that captures this task.", 
              "plan": "\nCome up with a sequence of steps to complete this task."}
json_path = "./blocksworld/blocksworld_hard_nl.json"

In [14]:
# First prompt agents v, c
task_agents = {'v': agent_v, 'c': agent_c}
responses = {}
z_query = ""
ctxt = [agent_v_context, agent_c_context]
letters = ['vowels', 'consonants']
for idx, p_query in query_iterator(json_path):
    
    for agent_num, (label, agent) in enumerate(task_agents.items(), start=1):
        query = p_query + "\n" + ctxt[agent_num-1] + task_query[query_mode] + f"The fact that I can only interact with {letters[agent_num-1]} should be reflected in the domain and problem definition."
        domain_out = f"./temp/{label}_domain{idx}.pddl"
        problem_out = f"./temp/{label}_problem{idx}.pddl"
        d, p = query_to_pddl(agent, query, domain_out, problem_out)
        responses[label] = (d, p)
        z_query += f"[DOMAIN{agent_num}]\n" + d + f"\n\n[PROBLEM{agent_num}]\n" + p + "\n\n"

    domain_out = f"./temp/z_domain{idx}.pddl"
    problem_out = f"./temp/z_problem{idx}.pddl"
    d_final, p_final = query_to_pddl(agent_z, z_query, domain_out, problem_out)
    ff_out = run_ff(domain_out, problem_out, ff_loc)

In [15]:
print(responses)
print(d_final)
print(p_final)
print(ff_out)

{'v': ('(define (domain blockstacking)\n  (:requirements :strips :equality)\n  (:predicates \n    (clear ?x)\n    (on-table ?x)\n    (on ?x ?y)\n    (holding ?x)\n    (hand-empty)\n    (vowel ?x)\n  )\n  \n  (:action pick-up\n    :parameters (?x)\n    :precondition (and (vowel ?x) (clear ?x) (on-table ?x) (hand-empty))\n    :effect (and (holding ?x) (not (clear ?x)) (not (on-table ?x)) (not (hand-empty)))\n  )\n\n  (:action put-down\n    :parameters (?x)\n    :precondition (holding ?x)\n    :effect (and (clear ?x) (hand-empty) (on-table ?x) (not (holding ?x)))\n  )\n\n  (:action stack\n    :parameters (?x ?y)\n    :precondition (and (vowel ?x) (vowel ?y) (clear ?y) (holding ?x))\n    :effect (and (on ?x ?y) (clear ?x) (hand-empty) (not (holding ?x)) (not (clear ?y)))\n  )\n\n  (:action unstack\n    :parameters (?x ?y)\n    :precondition (and (vowel ?x) (vowel ?y) (on ?x ?y) (clear ?x) (hand-empty))\n    :effect (and (holding ?x) (clear ?y) (not (on ?x ?y)) (not (clear ?x)) (not (hand-e

In [4]:
domain_out = "./temp/z_domain100.pddl"
problem_out = "./temp/z_problem100.pddl"
ff_out = run_ff(domain_out, problem_out, ff_loc)
print(ff_out)


ff: parsing domain file
domain 'COMBINED-BLOCKSTACKING' defined
 ... done.
ff: parsing problem file
problem 'COMBINED-BLOCK-ARRANGEMENT' defined
 ... done.


ff: goal can be simplified to FALSE. No plan will solve it


